# Personalization Agent Demo

## Connect to the Weaviate Cloud instance

In [1]:
import weaviate
from weaviate.classes.init import Auth
import os
from dotenv import load_dotenv

weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_key = os.getenv("WEAVIATE_API_KEY")
load_dotenv()

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url, auth_credentials=Auth.api_key(weaviate_key)
)

## Connect to the Personalization Agent

You can initialize the Personalization Agent, or connect to an existing one, as shown below.

In [2]:
from weaviate.agents.personalization import PersonalizationAgent
from weaviate.classes.config import DataType
from helpers import PA_DEMO_COLLECTION

if PersonalizationAgent.exists(client, PA_DEMO_COLLECTION):
    pa = PersonalizationAgent.connect(client=client, reference_collection=PA_DEMO_COLLECTION)
else:
    pa = PersonalizationAgent.create(
        client=client,
        reference_collection=PA_DEMO_COLLECTION,
        # These properties will be used to give the persona a "baseline" set of characteristics
        user_properties={
            "age": DataType.NUMBER,
            "favorite_genres": DataType.TEXT_ARRAY,
            "favorite_years": DataType.NUMBER_ARRAY,
            "language": DataType.TEXT,
        },
    )

## Create a persona

In [3]:
from weaviate.agents.classes import Persona
from weaviate.util import generate_uuid5
from uuid import uuid4  # If you want to generate a random UUID

persona_id = generate_uuid5("sebawita")  # To generate a deterministic UUID
# persona_id = uuid4()  # To generate a random UUID

# You can delete a persona if you want to remove it from the system
pa.delete_persona(persona_id)

if pa.has_persona(persona_id):
    print(f"Persona with ID {persona_id} already exists.")
else:
    print(f"Creating new persona with ID {persona_id}.")
    pa.add_persona(
        Persona(
            persona_id=persona_id,
            properties={
                "age": 30,
                "favorite_genres": ["Sci-Fi", "Horror", "Mystery"],
                "favorite_years": [1999, 1996, 2008, 2019],
                "language": "English",
            },
        )
    )

Creating new persona with ID 3df23b31-0e9d-515b-9aef-ed704641bbe2.


In [4]:
pa.get_persona(persona_id)

Persona(persona_id=UUID('3df23b31-0e9d-515b-9aef-ed704641bbe2'), properties={'age': 30.0, 'language': 'English', 'favorite_genres': ['Sci-Fi', 'Horror', 'Mystery'], 'favorite_years': [1999.0, 1996.0, 2008.0, 2019.0]})

## Add interactions

This is how the agent learns each persona's preferences. (At least one interaction is required to perform queries.)

In [ ]:
from weaviate.agents.classes import PersonaInteraction
from weaviate.collections.classes.filters import Filter
from helpers import get_movie_uuid  # Helper to get the UUID of a movie

# Need at least one interaction to get recommendations
pa.add_interactions(interactions=[
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Avatar"), weight=0.5
    ),
])

Fetched movie 'Avatar' from the collection


## Queries

We can already perform queries.

### Basic queries

Fast, most basic personalized queries

- Uses vectors of interaction history only

In [6]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=False)

In [8]:
from helpers import print_movie_response_details

print_movie_response_details(response, 5)

*****0*****
Avatar
['Action', 'Adventure', 'Fantasy', 'Science Fiction']
original rank: 0, personalized rank: None
*****1*****
Avatar 2
['Action', 'Adventure', 'Science Fiction', 'Fantasy']
original rank: 1, personalized rank: None
*****2*****
Avatar: Creating the World of Pandora
['Documentary']
original rank: 2, personalized rank: None
*****3*****
The Last Airbender
['Action', 'Adventure', 'Fantasy']
original rank: 3, personalized rank: None
*****4*****
The Invincible Iron Man
['Action', 'Adventure', 'Animation', 'Fantasy', 'Science Fiction']
original rank: 4, personalized rank: None


### Agent reranking

The agent can smartly rerank the results based on the information about the persona, as well as the interactions.

In [9]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=True)

print_movie_response_details(response, 5)

Ranking rationale: We've highlighted movies with genres like Action, Adventure, and Fantasy since they match your interests in Sci-Fi and similar genres. Animated and superhero-like themes were also given priority, echoing your past interactions.
*****0*****
Avatar
['Action', 'Adventure', 'Fantasy', 'Science Fiction']
original rank: 0, personalized rank: 0
*****1*****
Avatar 2
['Action', 'Adventure', 'Science Fiction', 'Fantasy']
original rank: 1, personalized rank: 1
*****2*****
Shang-Chi and the Legend of the Ten Rings
['Action', 'Adventure', 'Fantasy']
original rank: 11, personalized rank: 2
*****3*****
Raya and the Last Dragon
['Family', 'Fantasy', 'Animation', 'Action', 'Adventure']
original rank: 8, personalized rank: 3
*****4*****
Mulan
['Adventure', 'Fantasy', 'Drama']
original rank: 19, personalized rank: 4


### With Reranker + Instruction

- The most complex personalized queries
- Uses vectors of interaction history and AI-based reranker
- Instructions used to guide the reranker

In [ ]:
response = pa.get_objects(
    persona_id,
    limit=50,
    use_agent_ranking=True,
    instruction="I'm looking for something for the whole family, maybe historical."
)

print_movie_response_details(response, 5)

Ranking rationale: Since you're interested in family-friendly content with a historical backdrop, we've recommended movies that blend those elements. Historical fantasy movies and those involving well-loved characters are highlighted for a balance of adventure and nostalgia.
*****0*****
Mulan: Rise of a Warrior
['Adventure', 'Drama', 'Action']
original rank: 47, personalized rank: 1
*****1*****
The Pirates: The Last Royal Treasure
['Action', 'Adventure', 'Comedy', 'History']
original rank: 26, personalized rank: 2
*****2*****
The Myth
['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy']
original rank: 9, personalized rank: 3
*****3*****
Mulan
['Adventure', 'Fantasy', 'Drama']
original rank: 19, personalized rank: 4
*****4*****
Fairy Tail: Dragon Cry
['Action', 'Adventure', 'Comedy', 'Fantasy', 'Animation']
original rank: 49, personalized rank: 5


### Add more interactions

Over time, you will add more interactions to the agent, which will help it learn more about the persona's preferences.

Note each interaction can be positive or negative. 
(1: most positive, 0: neutral, -1: most negative)

In [13]:
interactions = [
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "How It Ends"), weight=0.7
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Doctor Sleep"), weight=0.6
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "The Howling"), weight=0.5
    ),
    PersonaInteraction(
        persona_id=persona_id,
        item_id=get_movie_uuid(client, "A Nightmare on Elm Street"),
        weight=0.9,
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "High Life"), weight=0.75
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Magic Mike"), weight=-0.9
    ),  # negative
]

pa.add_interactions(interactions=interactions)

Fetched movie 'How It Ends' from the collection
Fetched movie 'Doctor Sleep' from the collection
Fetched movie 'The Howling' from the collection
Fetched movie 'A Nightmare on Elm Street' from the collection
Fetched movie 'High Life' from the collection
Fetched movie 'Magic Mike' from the collection


### Retry queries

In [14]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=False)

print_movie_response_details(response, 5)

*****0*****
Son
['Horror', 'Thriller']
original rank: 0, personalized rank: None
*****1*****
Avatar
['Action', 'Adventure', 'Fantasy', 'Science Fiction']
original rank: 1, personalized rank: None
*****2*****
Slumber
['Horror']
original rank: 2, personalized rank: None
*****3*****
Avatar 2
['Action', 'Adventure', 'Science Fiction', 'Fantasy']
original rank: 3, personalized rank: None
*****4*****
Come True
['Science Fiction', 'Horror']
original rank: 4, personalized rank: None


### With Reranker + Instruction + Filter

- The most complex personalized queries
- Uses vectors of interaction history and AI-based reranker
- Instructions used to guide the reranker
- Filters out items that are not relevant to the user

In [15]:
# With Reranker + Instruction + Filter
response = pa.get_objects(
    persona_id,
    limit=50,
    use_agent_ranking=True,
    instruction="User's favorite sub genre of horror is slasher and least favorite is space horror",
    filters=Filter.by_property("genres").contains_any(["Mystery", "Horror"]),
)

In [16]:
print_movie_response_details(response, 10)

Ranking rationale: Based on your love for horror, particularly the slasher sub-genre, we've spotlighted films that closely match those interests while minimizing emphasis on space horror.
*****0*****
Son
['Horror', 'Thriller']
original rank: 0, personalized rank: 1
*****1*****
Slumber
['Horror']
original rank: 2, personalized rank: 0
*****2*****
Insidious
['Horror', 'Thriller']
original rank: 40, personalized rank: 2
*****3*****
Peninsula
['Action', 'Horror', 'Thriller']
original rank: 13, personalized rank: 3
*****4*****
Dark Light
['Horror', 'Science Fiction', 'Thriller']
original rank: 6, personalized rank: 4
*****5*****
Come True
['Science Fiction', 'Horror']
original rank: 4, personalized rank: 5
*****6*****
The Final
['Thriller', 'Horror']
original rank: 36, personalized rank: 6
*****7*****
Demonic
['Horror', 'Thriller']
original rank: 32, personalized rank: 7
*****8*****
Aftermath
['Horror', 'Crime', 'Drama', 'Thriller']
original rank: 48, personalized rank: 8
*****9*****
The Un

## Compare results 

What happens if we look at results across different query methods?

In [17]:
from helpers import compare_genre_match_scores

query = pa.query(persona_id=persona_id, strength=0.95)  # fully personalized reranking
query2 = pa.query(persona_id=persona_id, strength=0.0)  # no reranking

response1 = query.hybrid("slasher", limit=50)
response2 = query2.hybrid("slasher", limit=50)

compare_genre_match_scores(
    [response1, response2],
    preferred_genres={"Science Fiction", "Horror", "Mystery"},
    top_n=10,
    response_labels=["Personalized", "No Personalization"],
)

Rank Personalized Title                      Personalized Genres                     Score   | No Personalization Title                No Personalization Genres               Score  
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1    The Final                               Thriller, Horror                        1       | Chucky's Vacation Slides                Comedy, Horror                          1      
2    The Fanatic                             Crime, Thriller                         0       | Prom Night                              Horror, Thriller                        1      
3    Urban Legend                            Horror, Thriller                        1       | Monster                                 Crime, Drama                            0      
4    You're Next                             Horror, Thriller, Mystery               2  

We see that the personalized results are much more relevant to the user than the non-personalized results.